# git clone data

In [1]:
!git clone https://github.com/huijunam/CJ_AI_hackathon.git

fatal: destination path 'CJ_AI_hackathon' already exists and is not an empty directory.


In [2]:
import pandas as pd

chatbot_data_1 = pd.read_excel('/content/CJ_AI_hackathon/data/8f3202096f1387c5.xlsx')
chatbot_data_2 = pd.read_excel('/content/CJ_AI_hackathon/data/ad8cb80ea8623bbe.xlsx')

In [3]:
chatbot_data_1.sample(n =10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
19153,미국은 저렇게 내각구성을할때 과정을보여주는게 부럽다...,슬픔,NaN,NaN,NaN,NaN,NaN
15812,언제부터 경찰청장이 정치질 하는 정치인 이었냐??,분노,NaN,NaN,NaN,NaN,NaN
1537,중국이 싫다고 떳떳하게 말할수 있는 한국인들이 대체 몇%일까요?,공포,NaN,NaN,NaN,NaN,NaN
13058,저것들이 더 화나!,분노,NaN,NaN,NaN,NaN,NaN
5021,첫연애여서 정말모르겟어요ㅠ,공포,NaN,NaN,NaN,NaN,NaN
28751,좋은 음악 만들어주고 불러줘서 고마워 ♥,행복,NaN,NaN,NaN,NaN,NaN
27569,천상여자 같은 느낌이야.,행복,NaN,NaN,NaN,NaN,NaN
25019,서인국이나온다면 다들 봐야한다고 추천해주던데 이쯤되면 톱스타아님?,중립,NaN,NaN,NaN,NaN,NaN
27614,서로 의식하게 되고 좋아하게 된 시기가 겹쳐서 너무 신기해요,행복,NaN,NaN,NaN,NaN,NaN
27159,건강한 하루요~~~,행복,NaN,NaN,NaN,NaN,NaN


In [4]:
chatbot_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sentence    38594 non-null  object 
 1   Emotion     38594 non-null  object 
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   공포          7 non-null      object 
 6   5468        7 non-null      float64
dtypes: float64(4), object(3)
memory usage: 2.1+ MB


In [5]:
chatbot_data_1.describe()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,5468
count,0.0,0.0,0.0,7.000000
mean,NaN,NaN,NaN,10245.714286
std,NaN,NaN,NaN,12506.947639
min,NaN,NaN,NaN,4830.000000
25%,NaN,NaN,NaN,5348.000000
50%,NaN,NaN,NaN,5665.000000
75%,NaN,NaN,NaN,5967.500000
max,NaN,NaN,NaN,38594.000000


In [6]:
chatbot_data_1.count()

Sentence      38594
Emotion       38594
Unnamed: 2        0
Unnamed: 3        0
Unnamed: 4        0
공포                7
5468              7
dtype: int64

In [7]:
# 필요없는 열 삭제
chatbot_data_1= chatbot_data_1.iloc[:,:2]
chatbot_data_1

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [8]:
chatbot_data_1.isnull().sum() # 결측치 확인

Sentence    0
Emotion     0
dtype: int64

In [9]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

chatbot_data_1['Emotion'].value_counts()

행복    6037
놀람    5898
분노    5665
공포    5468
혐오    5429
슬픔    5267
중립    4830
Name: Emotion, dtype: int64

In [10]:
chatbot_data_2.sample(n =10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,행복,중립,슬픔,공포,혐오,분노,놀람
46726,S,아니 왠 술을 이렇게 마셔요 요새? 네 여보?!!,놀람,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10157,NaN,당연히 3번이죠.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19737,NaN,이거 큰일이네. 고혈압이면 어떡하지?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17204,NaN,내가 재수없는 놈인 거 알지? 근데 아주 재수없기만한 건 아닌가봐. 그래두 형을 만...,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49174,NaN,그건 있지. 내가 거짓말 한 거야. 나도 있지. 말하는 게 서투르니까. 사실은 너랑...,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12652,NaN,"아니야. 와, 예쁘다, 그랬잖아.",중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46140,NaN,동생!,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44461,NaN,"하 참, 그것만 안해본줄 알았더니 키스도 안해봤어?",놀람,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19507,S,여보세요? 지금 어디에 있어요?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20822,NaN,견적을 내기 위해 지역 회사들에게 연락해 보셨나요?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
chatbot_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55629 entries, 0 to 55628
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10001 non-null  object 
 1   Unnamed: 1  55628 non-null  object 
 2   Unnamed: 2  55622 non-null  object 
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   행복          1 non-null      float64
 6   중립          0 non-null      float64
 7   슬픔          0 non-null      float64
 8   공포          0 non-null      float64
 9   혐오          0 non-null      float64
 10  분노          0 non-null      float64
 11  놀람          0 non-null      float64
dtypes: float64(9), object(3)
memory usage: 5.1+ MB


In [12]:
chatbot_data_2.isnull().sum() # 결측치 확인

Unnamed: 0    45628
Unnamed: 1        1
Unnamed: 2        7
Unnamed: 3    55629
Unnamed: 4    55629
행복            55628
중립            55629
슬픔            55629
공포            55629
혐오            55629
분노            55629
놀람            55629
dtype: int64

In [13]:
chatbot_data_2 = chatbot_data_2.iloc[1:, :3] # 필요없는 열 삭제
chatbot_data_2

,Unnamed: 0,Unnamed: 1,Unnamed: 2
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,NaN,손님 왔어요.,중립
4,NaN,손님? 누구?,중립
5,NaN,몰라요. 팀장님 친구래요.,중립
...,...,...,...
55624,S,얘긴 다 끝났냐? 원예부,중립
55625,NaN,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,NaN,여자 숨겨달라는거면 사절이다.,중립
55627,NaN,아무래도 안되나요?,중립


In [14]:
chatbot_data_2[chatbot_data_2['Unnamed: 0'] =='S']

,Unnamed: 0,Unnamed: 1,Unnamed: 2
1,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
8,S,그래서... 무슨 일 해?,중립
16,S,언제부터 시작할 수 있어?,중립
30,S,안 내켜.,혐오
42,S,"집 짓는 이유가 그렇게 중요해? 왜, 내가 사기라도 칠까봐?",혐오
...,...,...,...
55604,S,자네는 대체 뭘 하러 왔나! 젖은 생쥐 꼴이 된 나를 보면서 비웃으러 왔나?,분노
55609,S,조사결과에 따르면 의뢰인의 아버지는 횡령같은 걸 하지 않았다고 하는군.,중립
55614,S,그게 무슨 소리야?,중립
55619,S,기억해 두게. 이 사건에서 사실은 필요치 않아.,중립


In [15]:
chatbot_data_2 = chatbot_data_2.iloc[:,1:] # Unnamed: 0 열 삭제
chatbot_data_2

,Unnamed: 1,Unnamed: 2
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [16]:
chatbot_data_2['Unnamed: 2'].value_counts()

중립     43786
놀람      4866
분노      3628
슬픔      1972
행복      1030
혐오       220
공포        98
ㅍ         12
분          4
ㅈ중립        1
분ㄴ         1
중림         1
ㄴ중립        1
줄          1
Name: Unnamed: 2, dtype: int64

In [17]:
chatbot_data_2[chatbot_data_2['Unnamed: 2'] == 'ㅍ'] # 확인

,Unnamed: 1,Unnamed: 2
25037,인터넷에다 팔아야 되나.. 근데 너 이제 맘 잡았나보다?,ㅍ
27177,일부러 그럴 리가 있어요?,ㅍ
27948,그럼요?,ㅍ
28090,샤프심이 없어서..가지러 왔다가 누나 보이길래..,ㅍ
28455,아~ 자기소개서 쓰기 힘들어 죽겠어요~,ㅍ
28482,원래 남자들..다른 남자랑 친하게 지내면 그러니까..,ㅍ
36684,"그럼, 임마! 내가 누구냐? 한번 실수는 병가지상사다~이러시면서 나의 인간성과 품위...",ㅍ
37261,"처리 할거야. 처리해야지, 그게 돈이 얼만데?",ㅍ
37428,"와, 어디서 꽁돈이라도 생겼나?",ㅍ
37864,"엄마, 엄만 결혼이 뭐 그리 좋아?",ㅍ


In [18]:
# 90 이하인 카테고리 추출
value_counts = chatbot_data_2['Unnamed: 2'].value_counts()
categories_to_remove = value_counts[value_counts <= 90].index

# 해당 카테고리를 포함하지 않는 행만 선택
chatbot_data_2 = chatbot_data_2[~chatbot_data_2['Unnamed: 2'].isin(categories_to_remove)]

In [19]:
chatbot_data_2['Unnamed: 2'].value_counts()

중립    43786
놀람     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
공포       98
Name: Unnamed: 2, dtype: int64

In [20]:
chatbot_data_2.columns =['Sentence', 'Emotion']

In [21]:
chatbot_data_2

,Sentence,Emotion
1,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
2,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [22]:
chatbot_data_1['Emotion'].unique()

array(['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오'], dtype=object)

In [23]:
chatbot_data_2['Emotion'].unique()
chatbot_data_2 = chatbot_data_2.dropna(how='any')# Null 값 제거

array(['분노', '혐오', '중립', '놀람', '행복', '공포', '슬픔', nan], dtype=object)

In [24]:
chatbot_data_2['Emotion'].value_counts()

중립    43786
놀람     4866
분노     3628
슬픔     1972
행복     1030
혐오      220
공포       98
Name: Emotion, dtype: int64

In [25]:
chatbot_data_2[chatbot_data_2['Emotion'] == '중립'] # 확인

,Sentence,Emotion
3,손님 왔어요.,중립
4,손님? 누구?,중립
5,몰라요. 팀장님 친구래요.,중립
6,내 친구? 친구 누구?,중립
8,그래서... 무슨 일 해?,중립
...,...,...
55624,얘긴 다 끝났냐? 원예부,중립
55625,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55626,여자 숨겨달라는거면 사절이다.,중립
55627,아무래도 안되나요?,중립


In [26]:
len(chatbot_data_1), len(chatbot_data_2)

(38594, 55607)

In [27]:
df_data = pd.concat([chatbot_data_1,chatbot_data_2], ignore_index=True)
df_data # 최종 데이터

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
94196,얘긴 다 끝났냐? 원예부,중립
94197,"예. 그거 때문에, 부탁이 있......는......데요.",중립
94198,여자 숨겨달라는거면 사절이다.,중립
94199,아무래도 안되나요?,중립


In [28]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94201 entries, 0 to 94200
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  94200 non-null  object
 1   Emotion   94194 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


# 데이터셋 전처리

In [30]:
target_classes ={
    '공포': 0,
    '놀람': 1,
    '분노': 2,
    '슬픔': 3,
    '중립': 4,
    '행복': 5,
    '혐오': 6
  }

In [31]:
# 레이블링
for cls in target_classes:
  df_data.loc[(df_data['Emotion'] == cls), 'Emotion'] = target_classes[cls]

<ipython-input-31-525f87733a11>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data.loc[(df_data['Emotion'] == cls), 'Emotion'] = target_classes[cls]


In [32]:
data_list =[]
# ['sentence', 'class'] 형태로 변환
for q, label in zip(df_data['Sentence'], df_data['Emotion']):
  data =[]
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [33]:
print(data_list[6000])

['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']


# wandb 설정

In [34]:
# wandb 설치
!pip install wandb

In [35]:
!wandb login 09b76e2d7910cb85b58be240c8cf088ff601d126

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [36]:
import wandb
wandb.init(project="Team-Project", entity='huijunam')

wandb: Currently logged in as: huijunam. Use `wandb login --relogin` to force relogin


# 환경 설정 & KoBERT 설치
- https://github.com/ChangZero/koBERT-finetuning-demo/blob/main/kobert_colab.ipynb

In [37]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a) # 수정한 부분
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [39]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [40]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [41]:
# 하이퍼 파라미터 설정
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# # Setting parameters
# max_len =64
# batch_size =64
# warmup_ratio =0.1
# num_epochs =5
# max_grad_norm =1
# log_interval =200
# learning_rate =5e-5

In [42]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [43]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [44]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [45]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 23)

In [46]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [47]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [48]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 감정 클래스 수로 조정 긍정/부정 클래스2개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [50]:
# torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)
device

device(type='cuda')

In [51]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

In [52]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [53]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 모델 학습

In [54]:
import time

train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train() #train

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        start_time = time.time() # time
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval() # inference
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    end_time = time.time()
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    print("Time: {:2.f}sec".format((end_time - start_time)))
    test_history.append(test_acc / (batch_id+1))


  0%|          | 0/1178 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.022761821746826 train acc 0.125
epoch 1 batch id 201 loss 1.4183768033981323 train acc 0.45149253731343286
epoch 1 batch id 401 loss 1.0094115734100342 train acc 0.5278600374064838
epoch 1 batch id 601 loss 0.9834405183792114 train acc 0.5664257487520798
epoch 1 batch id 801 loss 0.8229309320449829 train acc 0.5871566791510612
epoch 1 batch id 1001 loss 0.8152716755867004 train acc 0.6017420079920079
epoch 1 train acc 0.6116421626422688


  0%|          | 0/295 [00:00<?, ?it/s]

epoch 1 test acc 0.6734570744288872


  0%|          | 0/1178 [00:00<?, ?it/s]

...

epoch 2 test acc 0.6895817980840088


  0%|          | 0/1178 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.8978694677352905 train acc 0.71875
epoch 3 batch id 201 loss 0.807506263256073 train acc 0.740360696517413
epoch 3 batch id 401 loss 0.6510478258132935 train acc 0.7511299875311721
epoch 3 batch id 601 loss 0.4629998803138733 train acc 0.759619384359401
epoch 3 batch id 801 loss 0.4530802369117737 train acc 0.7656445068664169
epoch 3 batch id 1001 loss 0.5799936652183533 train acc 0.7693556443556444
epoch 3 train acc 0.7708397197069735


  0%|          | 0/295 [00:00<?, ?it/s]

epoch 3 test acc 0.690694086219602


In [55]:
torch.save(model.state_dict(), "./model_state_dict.pt")

In [ ]:
import argparse # for wandb

# 키워드 추출
- 명사, 동사, 형용사

In [58]:
!pip install konlpy

from konlpy.tag import Okt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.5 MB/s eta 0:00:00


In [59]:
okt = Okt()
kor_sentence = '너무 배고파 ㅜㅜ'
print('OKT 품사 태깅 :', okt.pos(kor_sentence))

OKT 품사 태깅 : [('너무', 'Adverb'), ('배고파', 'Adjective'), ('ㅜㅜ', 'KoreanParticle')]


In [60]:
# 명사, 동사, 형용사 추출하기
keyword = [x for (x, y) in okt.pos(kor_sentence) if y in ['Verb', 'Noun', 'Adjective']]
print('\nOKT 명사, 동사, 형용사 추출 :', keyword)


OKT 명사, 동사, 형용사 추출 : ['배고파']


In [ ]:
# okt.normalize(kor_sentence)

# 새로운 문장 테스트

In [ ]:
# target_classes ={
#     '공포': 0,
#     '놀람': 1,
#     '분노': 2,
#     '슬픔': 3,
#     '중립': 4,
#     '행복': 5,
#     '혐오': 6
#   }

In [57]:
reverse =dict(map(reversed, target_classes.items()))
print(reverse)

{0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}


In [93]:
# 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

def predict_and_keword(predict_sentence):
  new_data =[[predict_sentence, '0']]
  # print(new_data)

  new_dataset = BERTDataset(new_data, 0, 1, tokenizer, vocab ,max_len, True, False)
  new_dataloader = torch.utils.data.DataLoader(new_dataset, batch_size = batch_size, num_workers =5)

  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(new_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    test_eval =[]
    for logit in out:
      test_eval.append(reverse[np.argmax(logit.detach().cpu().numpy())])

    # 키워드 추출: 명사, 동사, 형용사 추출하기
    keyword = [x for (x, y) in okt.pos(predict_sentence) if y in ['Verb', 'Noun', 'Adjective']]
    print('\nOKT 명사, 동사, 형용사 추출 :', keyword)
    test_eval += keyword

    return test_eval

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [95]:
predict_and_keword('어제 헤어졌어 ㅜ')

  0%|          | 0/1 [00:00<?, ?it/s]


OKT 명사, 동사, 형용사 추출 : ['어제', '헤어졌어']


['슬픔', '어제', '헤어졌어']